# 케라스로 전이 학습하기

이 노트북에서 사전 훈련된 모델(VGGNet19)을 적재하고 핫도그를 감지하는 새 작업에 미세 조정하는 방법을 다룹니다.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rickiepark/dl-illustrated/blob/master/notebooks/10-4.transfer_learning_in_keras.ipynb)

#### 라이브러리를 적재합니다.

In [1]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#### 사전 훈련된 VGG19 모델을 적재합니다.

In [2]:
vgg19 = VGG19(include_top=False,
              weights='imagenet',
              input_shape=(224,224,3),
              pooling=None)

80150528/80134624 [==============================] - 0s 0us/step


#### VGGNet19 모델의 모든 층을 동결합니다.

In [3]:
for layer in vgg19.layers:
    layer.trainable = False

#### 사용자 정의 분류 층을 추가합니다.

In [4]:
# 시퀀셜 모델을 만들고 VGG19 모델을 추가합니다.
model = Sequential()
model.add(vgg19)

# VGG19 모델 위에 새로운 층을 추가합니다. 
model.add(Flatten(name='flattened'))
model.add(Dropout(0.5, name='dropout'))
model.add(Dense(2, activation='softmax', name='predictions'))

#### 모델을 설정합니다.

In [5]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#### 훈련할 데이터를 준비합니다.

In [6]:
# 코랩을 사용할 경우 다음 명령을 실행하세요.
!wget https://git.io/JtRjg -O hot-dog-not-hot-dog.tar.gz

--2021-08-21 13:20:47--  https://git.io/JtRjg
Resolving git.io (git.io)... 34.239.30.50, 50.16.165.212, 44.194.107.75, ...
Connecting to git.io (git.io)|34.239.30.50|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/rickiepark/dl-illustrated/raw/master/notebooks/hot-dog-not-hot-dog.tar.gz [following]
--2021-08-21 13:20:47--  https://github.com/rickiepark/dl-illustrated/raw/master/notebooks/hot-dog-not-hot-dog.tar.gz
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rickiepark/dl-illustrated/master/notebooks/hot-dog-not-hot-dog.tar.gz [following]
--2021-08-21 13:20:47--  https://raw.githubusercontent.com/rickiepark/dl-illustrated/master/notebooks/hot-dog-not-hot-dog.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 18

In [7]:
!rm -rf hot-dog-not-hot-dog
!tar -xvzf hot-dog-not-hot-dog.tar.gz

hot-dog-not-hot-dog/
hot-dog-not-hot-dog/train/
hot-dog-not-hot-dog/train/not_hot_dog/
hot-dog-not-hot-dog/train/not_hot_dog/232844.jpg
hot-dog-not-hot-dog/train/not_hot_dog/223676.jpg
hot-dog-not-hot-dog/train/not_hot_dog/146029.jpg
hot-dog-not-hot-dog/train/not_hot_dog/211973.jpg
hot-dog-not-hot-dog/train/not_hot_dog/96213.jpg
hot-dog-not-hot-dog/train/not_hot_dog/824157.jpg
hot-dog-not-hot-dog/train/not_hot_dog/126784.jpg
hot-dog-not-hot-dog/train/not_hot_dog/130384.jpg
hot-dog-not-hot-dog/train/not_hot_dog/222138.jpg
hot-dog-not-hot-dog/train/not_hot_dog/110171.jpg
hot-dog-not-hot-dog/train/not_hot_dog/776077.jpg
hot-dog-not-hot-dog/train/not_hot_dog/204169.jpg
hot-dog-not-hot-dog/train/not_hot_dog/166821.jpg
hot-dog-not-hot-dog/train/not_hot_dog/766638.jpg
hot-dog-not-hot-dog/train/not_hot_dog/816725.jpg
hot-dog-not-hot-dog/train/not_hot_dog/180453.jpg
hot-dog-not-hot-dog/train/not_hot_dog/250720.jpg
hot-dog-not-hot-dog/train/not_hot_dog/183313.jpg
hot-dog-not-hot-dog/train/not_ho

In [8]:
# 두 개의 이미지 제너레이터 객체를 만듭니다.
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last',
    rotation_range=30,
    horizontal_flip=True,
    fill_mode='reflect')

valid_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last')

In [9]:
# 배치 크기를 설정합니다.
batch_size=32

In [10]:
# 훈련과 검증 제너레이터를 만듭니다.
train_generator = train_datagen.flow_from_directory(
    directory='./hot-dog-not-hot-dog/train',
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

valid_generator = valid_datagen.flow_from_directory(
    directory='./hot-dog-not-hot-dog/test',
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

Found 498 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [11]:
model.fit(train_generator, steps_per_epoch=15, 
          epochs=16, validation_data=valid_generator, 
          validation_steps=15)

Epoch 1/16
15/15 [==============================] - 48s 1s/step - loss: 0.9145 - accuracy: 0.5901 - val_loss: 0.5617 - val_accuracy: 0.7208
Epoch 2/16
15/15 [==============================] - 9s 615ms/step - loss: 0.5779 - accuracy: 0.7403 - val_loss: 0.4828 - val_accuracy: 0.7729
Epoch 3/16
15/15 [==============================] - 9s 634ms/step - loss: 0.4820 - accuracy: 0.7583 - val_loss: 0.4250 - val_accuracy: 0.7917
Epoch 4/16
15/15 [==============================] - 9s 623ms/step - loss: 0.5577 - accuracy: 0.7618 - val_loss: 0.4548 - val_accuracy: 0.7812
Epoch 5/16
15/15 [==============================] - 9s 616ms/step - loss: 0.4242 - accuracy: 0.8197 - val_loss: 0.6012 - val_accuracy: 0.7563
Epoch 6/16
15/15 [==============================] - 9s 625ms/step - loss: 0.4188 - accuracy: 0.8262 - val_loss: 0.4477 - val_accuracy: 0.8062
Epoch 7/16
15/15 [==============================] - 9s 639ms/step - loss: 0.4075 - accuracy: 0.8498 - val_loss: 0.4234 - val_accuracy: 0.8083
Epoch 8/